In [1]:
%matplotlib inline
%run ../../path_import.py
from plots import *
from training_reparam import *
from eubo import *
from model_reparam import *

import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
_, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 20
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 300
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-pr-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)

DEVICE = torch.device('cuda:0')

In [3]:
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE)
if CUDA:
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
RESAMPLE = True
DETACH=True
train_cfz_reparam(Eubo_cfz_pr_eta, enc_eta, enc_z, optimizer, Data, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE, RESAMPLE=RESAMPLE, DETACH=DETACH)

epoch=0, symKL=6068.922, EUBO=-522.964, ELBO=-6996.337, ESS=1.385 (71s)
epoch=1, symKL=5521.233, EUBO=-388.177, ELBO=-6314.168, ESS=1.385 (72s)
epoch=2, symKL=5051.075, EUBO=-254.115, ELBO=-5710.090, ESS=1.389 (71s)
epoch=3, symKL=4643.481, EUBO=-144.379, ELBO=-5193.337, ESS=1.390 (72s)
epoch=4, symKL=4308.768, EUBO=-50.763, ELBO=-4764.581, ESS=1.397 (73s)
epoch=5, symKL=4023.451, EUBO=47.775, ELBO=-4380.592, ESS=1.401 (64s)
epoch=6, symKL=3779.967, EUBO=122.889, ELBO=-4062.507, ESS=1.407 (61s)
epoch=7, symKL=3577.216, EUBO=200.134, ELBO=-3783.496, ESS=1.416 (76s)
epoch=8, symKL=3389.409, EUBO=239.983, ELBO=-3554.579, ESS=1.424 (86s)
epoch=9, symKL=3242.703, EUBO=289.412, ELBO=-3358.202, ESS=1.435 (87s)
epoch=10, symKL=3120.756, EUBO=326.557, ELBO=-3199.336, ESS=1.444 (85s)
epoch=11, symKL=2992.089, EUBO=343.879, ELBO=-3053.780, ESS=1.459 (87s)
epoch=12, symKL=2891.653, EUBO=365.731, ELBO=-2929.671, ESS=1.472 (82s)
epoch=13, symKL=2796.525, EUBO=379.470, ELBO=-2821.371, ESS=1.488 (86s)

In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
RESAMPLE= False
obs, q_eta, q_z, symkl_test, eubo_test, elbo_test = test(Eubo_cfz_pr_eta, enc_eta, enc_z, Data, K, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, RESAMPLE=RESAMPLE)

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap